<a href="https://colab.research.google.com/github/wadaka0821/nlp-tutorial/blob/main/questions/5_4_ngram_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-gram 言語モデル
## 作成者：和田
## 最終更新日：2023/03/09

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.7 MB/s eta 0:00:00


In [ ]:
import nltk
from datasets import load_dataset
import numpy as np
from copy import deepcopy
from tqdm import tqdm

In [ ]:
nltk.download('punkt') # 単語分割に必要なデータのダウンロード

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
dataset = load_dataset("common_gen") # 使用するデータのダウンロード

Generating train split:   0%|          | 0/67389 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4018 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1497 [00:00<?, ? examples/s]

Dataset common_gen downloaded and prepared to /root/.cache/huggingface/datasets/common_gen/default/2020.5.30/1a9e8bdc026c41ce7a9e96260debf7d2809cb7fd63fa02b017e4fac1b00c6b23. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
corpus = dataset['train']['target']

In [ ]:
corpus[:5]

['Skier skis down the mountain',
 'A skier is skiing down a mountain.',
 'Three skiers are skiing on a snowy mountain.',
 'The dog is wagging his tail.',
 'A dog wags his tail at the boy.']

In [ ]:
tokenized_corpus = map(nltk.tokenize.word_tokenize, corpus)

In [ ]:
class NgramModel():
    '''N-gram 言語モデル

    Attributes
    ----------
    tokenized_corpus : iterable | list[str] | list[list[str]]
        言語モデルの構築に使用するコーパス
        単語分割済みの時は list[list[str]] | iterable
        そうでないときは   list[str] | iterable
    save_corpus : bool
        引数のコーパスを保存するかどうか
    n : int
        正の整数
        n-gram の n の部分
    n_gram : dict[str, list[list[str] | ndarray]]
        p(w_2 | w_0, w_1) の場合
        w_0, w_1 が key
        w_2 とその出現頻度のペアが value
    
    See Also
    --------
    _build_model : コーパスから n_gram を構築
    get_p : 2語以下の単語のリストから次の単語とその確率を返す
    generate : 与えられた文字列から n_gram に従って文を生成
    '''
    NULL_TOKEN = '<NULL>'
    SOS_TOKEN = '<s>'
    EOS_TOKEN = '</s>'
    CONNECTION_TOKEN = '=='

    def __init__(self, tokenized_corpus, tokenized=True, save_corpus=False, n=3):
        '''

        Parameters
        ----------
        tokenized_corpus : iterable | list[str] | list[list[str]]
            言語モデルの構築に使用するコーパス
            単語分割済みの時は list[list[str]] | iterable
            そうでないときは   list[str] | iterable
        tokenized : bool
            単語分割済みかどうかを表す
        save_corpus : bool
            引数のコーパスを保存するかどうか
            今回の例のように map instance は一度 for ループに入れると消えるのでそれを保存する場合は True にする
        n : int
            正の整数
            n-gram の n の部分
        '''
        self.tokenized_corpus = tokenized_corpus
        self.save_corpus = save_corpus
        if not tokenized:
            self.tokenized_corpus = map(nltk.tokenize.word_tokenize, self.tokenized_corpus)
        if self.save_corpus:
            self.tokenized_corpus = list(self.tokenized_corpus)
        assert n > 0, f'argument n need to be positive integer. !!!{n=}!!!' # n が正の整数かチェック
        self.n = n
        self.n_gram = dict()

        print(f'start building n_gram model')
        self._build_model()
        print(f'\nfinish building n_gram model.')

    def _build_model(self):
        '''n_gram の構築
        '''
        for sent in tqdm(self.tokenized_corpus):
            sent = [self.SOS_TOKEN] + sent + [self.EOS_TOKEN]
            for i in range(1, len(sent)):
                n_word = [self.NULL_TOKEN for _ in range(self.n-i-1)] + sent[max(0, i-self.n+1):i]
                key = self.CONNECTION_TOKEN.join(n_word)
                if key in self.n_gram:
                    if sent[i] in self.n_gram[key][0]:
                        idx = self.n_gram[key][0].index(sent[i])
                        self.n_gram[key][1][idx] += 1
                    else:
                        self.n_gram[key][0].append(sent[i])
                        self.n_gram[key][1] = np.append(self.n_gram[key][1], 1.)
                else:
                    self.n_gram[key] = [[sent[i]], np.array([1.])]
        # コーパスを保存しない場合は None に置き換える
        if not self.save_corpus:
            self.tokenized_corpus = None

    def get_p(self, keys):
        '''2語以下の単語のリストから次の単語とその確率を返す
        
        Parameters:
        keys : list[str]
            予測したい単語の前の単語列
            長さ（単語数）は n-gram なら n-1 以下が必須
        '''
        assert len(keys) < self.n, f'this model is {self.n}-gram model. need len(keys) < {self.n}, but {len(keys)=}' # 入力された単語列の長さが n-1 より大きければエラー
        keys = [self.NULL_TOKEN for _ in range(self.n-len(keys)-1)] + keys
        keys = self.CONNECTION_TOKEN.join(keys)
        p = deepcopy(self.n_gram.get(keys))
        assert p, f'{keys=} not in this model' # key が存在しなければエラー
        p[1] /= np.sum(p[1]) # 確率に変換
        
        return p

    def generate(self, sent=list()):
        '''与えられた文字列から n_gram に従って文を生成

        Paramters
        ---------
        sent : list[str]
            生成途中の単語列
            get_p メソッドのような長さの制約はない
        '''
        sent = [self.NULL_TOKEN for _ in range(self.n-len(sent)-2)] + [self.SOS_TOKEN] + sent
        while sent[-1] != self.EOS_TOKEN: # EOS_TOKEN が生成されるまで単語を生成
            key = self.CONNECTION_TOKEN.join(sent[-self.n+1:])
            p = self.n_gram.get(key)
            assert p, f'key error'
            idx = np.argmax(p[1]) # 最大確率の単語を生成結果として採用
            sent.append(p[0][idx])
        ret_sent = [i for i in sent if i not in [self.NULL_TOKEN, self.SOS_TOKEN, self.EOS_TOKEN]] # 生成の過程で追加した NULL_TOKEN, SOS_TOKEN, EOS_TOKEN を取り除く
        return ret_sent
        

In [ ]:
model = NgramModel(tokenized_corpus)

start building n_gram model


67389it [00:19, 3408.68it/s]


finish building n_gram model.


In [ ]:
model.get_p(['<s>', 'moon'])

[['rising', 'rises', 'and', 'with', 'comes', 'has', 'in'],
 array([0.4 , 0.08, 0.28, 0.08, 0.08, 0.04, 0.04])]

In [ ]:
# 入力列の長さが n-1 より大きい場合
model.get_p(['<s>', 'moon', 'is'])

AssertionError: ignored

In [ ]:
# 存在しない key を指定した場合
model.get_p(['<s>', 'Japan'])

AssertionError: ignored

In [ ]:
# <s> から生成
model.generate()

['A', 'man', 'is', 'working', 'on', 'a', 'sunny', 'day']

In [ ]:
# 先頭から数文字だけ指定して生成
model.generate(['a', 'sun', 'is'])

['a',
 'sun',
 'is',
 'the',
 'only',
 'thing',
 'you',
 'need',
 'to',
 'know',
 'about',
 'the',
 'issue',
 '.']

In [ ]:
model.generate(['the', 'sun'])

['the', 'sun', 'rises', 'over', 'a', 'city']

## 問題1
---
文の生成を行う generateメソッドは最大確率のものを結果として出力しています．  
モデルから得られる確率に従ってランダムに生成するようにしてください．

## 問題2
---
1) 今回作成したモデルの generateメソッドは get_pメソッドとは違い，入力列が n-1 より小さくても文生成をすることが出来ます．それはなぜでしょうか？  
2) generateメソッドは get_pメソッドを使用していません．get_pを使用して全く同じ動作をするように，generateメソッドを書き換えてください．